<a href="https://colab.research.google.com/github/RapaDewan/CSE499B_Capstone/blob/main/CSE499B_Trying_BERT/CSE499B_Trying_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Replace 'path_to_dataset' with the actual path to your dataset in Google Drive
import pandas as pd
file_path = '/content/drive/MyDrive/Resume.csv'
df = pd.read_csv(file_path)

In [8]:
# Shuffle the dataset and take 20 samples
df = df.sample(frac=1, random_state=42).reset_index(drop=True)  # Shuffle the dataset
sample_df = df.head(20)  # Take the first 20 entries

In [15]:
sample_df.head()

,ID,Resume_str,Resume_html,Category
0,99244405,Kpandipou Koffi Summary ...,"<div class=""RNA skn-rbn1 fontsize fontface vma...",TEACHER
1,17562754,DIRECTOR OF DIGITAL TRANSFORMATION ...,"<div class=""fontsize fontface vmargins hmargin...",DIGITAL-MEDIA
2,30311725,SENIOR PROJECT MANAGER Professi...,"<div class=""fontsize fontface vmargins hmargin...",CONSTRUCTION
3,19007667,CHEF Summary Experienced ca...,"<div class=""fontsize fontface vmargins hmargin...",CHEF
4,11065180,OPERATIONS MANAGER Summary E...,"<div class=""fontsize fontface vmargins hmargin...",BANKING


In [10]:
# Display the sampled data
sample_df[['ID', 'Category']]

,ID,Category
0,99244405,TEACHER
1,17562754,DIGITAL-MEDIA
2,30311725,CONSTRUCTION
3,19007667,CHEF
4,11065180,BANKING
5,39237915,BUSINESS-DEVELOPMENT
6,17199951,DESIGNER
7,18236085,BUSINESS-DEVELOPMENT
8,79663360,TEACHER
9,62312955,DESIGNER


In [39]:
!pip install torch transformers numpy


In [40]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
import pandas as pd



In [48]:


# Step 3: Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

# Step 4: Define function to get embeddings
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze()

# Step 5: Define function to calculate similarity
def calculate_similarity(resume_text, job_desc):
    resume_embedding = get_embedding(resume_text)
    job_embedding = get_embedding(job_desc)
    similarity_score = torch.nn.functional.cosine_similarity(resume_embedding.unsqueeze(0), job_embedding.unsqueeze(0))
    return similarity_score.item()

# Step 6: Sample dataset (replace with your actual dataset)
sample_df = pd.DataFrame({
    'ID': range(1, 21),  # Ensure this has 20 samples
    'Resume_str': [
        "Software developer with experience in Python and Java.",
        "Data scientist skilled in machine learning and NLP.",
        "Web developer with a strong background in JavaScript.",
        "Full-stack developer with expertise in React and Node.js.",
        "Backend developer with experience in APIs and microservices.",
        "Data analyst with experience in SQL and Python.",
        "Business analyst with experience in process automation.",
        "Software engineer with a focus on mobile development.",
        "DevOps engineer experienced in CI/CD pipelines.",
        "Machine learning engineer skilled in TensorFlow and Keras.",
        "AI researcher with expertise in natural language processing.",
        "Cloud engineer with experience in AWS and Azure.",
        "Security analyst with strong background in cybersecurity.",
        "Project manager with experience in software development.",
        "Marketing specialist with experience in digital marketing.",
        "HR manager with expertise in recruitment processes.",
        "Quality assurance tester with experience in automation.",
        "Product manager with experience in agile methodologies.",
        "Customer support representative with experience in tech support.",
        "Network engineer with a focus on system integration."
    ]
})

# Example Job Description (replace with your actual job description)
job_description = "Looking for a Python developer with strong ML skills."

# Step 7: Calculate similarity scores
sample_df['Score'] = sample_df['Resume_str'].apply(lambda x: calculate_similarity(x, job_description))

# Step 8: Rank the resumes based on similarity scores
sample_df = sample_df.sort_values(by='Score', ascending=False).reset_index(drop=True)

# Step 9: Generate feedback based on similarity score
def generate_feedback(score):
    if score >= 0.85:
        return "This resume is highly suitable for the job based on the matching skills and experience."
    elif score >= 0.60:
        return "This resume has moderate relevance to the job description."
    else:
        return "This resume doesn't match well with the job description."

# Add feedback column based on similarity score
sample_df['Feedback'] = sample_df['Score'].apply(generate_feedback)

# Step 10: Display the ranked resumes with similarity scores and feedback
print(sample_df[['ID', 'Score', 'Feedback']])


    ID     Score                                           Feedback
0   19  0.871690  This resume is highly suitable for the job bas...
1   18  0.861176  This resume is highly suitable for the job bas...
2    4  0.855663  This resume is highly suitable for the job bas...
3   14  0.848871  This resume has moderate relevance to the job ...
4    5  0.846776  This resume has moderate relevance to the job ...
5    3  0.825272  This resume has moderate relevance to the job ...
6   20  0.824396  This resume has moderate relevance to the job ...
7   11  0.821874  This resume has moderate relevance to the job ...
8   16  0.820578  This resume has moderate relevance to the job ...
9   15  0.820179  This resume has moderate relevance to the job ...
10   6  0.815118  This resume has moderate relevance to the job ...
11  12  0.814921  This resume has moderate relevance to the job ...
12  17  0.814624  This resume has moderate relevance to the job ...
13   8  0.808002  This resume has moderate relev